# Comparison of different approaches to LMP setting

## Discussion
### General notes
- it is more or less clear how to "unpack" a network topology
- the IEEE 1888 RTS test case is [here](https://www2.ee.washington.edu/research/pstca/pf118/pg_tca118bus.htm). I.e. the [data](https://www2.ee.washington.edu/research/pstca/pf118/ieee118cdf.txt) are available, as well as [short](https://www2.ee.washington.edu/research/pstca/formats/cdf.txt) and [long](https://ieeexplore.ieee.org/document/4075293/) format descriptions. Note that there are some discrepancies between the two, but we'll manage, they are not terrible.

### What is **done** by the moment:
- a simple [data-parser](./IEEE_CDF_parser.jl), including a helper-function to unpack a Node-incidence matrix;
- a simple model for the 3-node case just to test everything works as expected;
   
### Methodology-related questions

Well, how exactly do we proceed?

### Data-related questions:
    - how do we cook up the missing data; most importantly, Pmax, Dmax and utilities? I can imagine we can assume for starters that $P_\max = P_\text{actual}$ from the data file and the same logic for the demand. Maybe later we'll want to assume it is, say 20% more. What do we do with utilities? Is there any common practice for that? or just ensure it is high enough?
    - are we planning to visualize the data somehow? I mean, put numbers on a real network scheme? Is there any standard for that in the field?
---

## Headers -- technical section

In [1]:
using JuMP
using Cbc 
using DataFrames

import IEEE_CDF_parser # my data file parser


## A 3-node test case
### Case description

In [6]:
# Cook up come simple test data
# three-bus system
buses = DataFrame(BusNo = [1,2,3], # bus number
                  Pmin = [0,0,0], # min supply
                  Pmax = [10,5,0], # max supply
                  Dmax = [0,0,7], # max demand
                  b = [0,0,50], # consumers' utility per kWh
                  C = [5,3,0], # costs per kWh
                  cSU = [0,0,0]) # startup costs

branches = DataFrame(Arc = ["a","b","c"], TapBusNo = [1, 2, 1], ZBusNo = [2,3,3], x=[0.1,0.1,0.1])


,Arc,TapBusNo,ZBusNo,x
1,a,1,2,0.1
2,b,2,3,0.1
3,c,1,3,0.1


Test case:
![A simple three-node examplez](img/3node.png)

In [3]:
# Make a node-arc incidence matrix

NIM = IEEE_CDF_parser.makeNIMatrix(size(buses,1), branches)

3×3 Array{Float64,2}:
 -1.0   0.0  -1.0
  1.0  -1.0   0.0
  0.0   1.0   1.0

### UC Model description

In [4]:
## ================== OPF model definition =====================
UC = JuMP.Model(solver=CbcSolver())
Nbus = size(buses,1)
Nbranches = size(branches,1)

# constants
const Fmax = 100
Pmax = maximum(buses[:,:Pmax])
Dmax = maximum(buses[:,:Dmax])

@variable(UC, -Fmax <= f[l=1:Nbranches] <= Fmax) # flow through each branch l \in Branches
@variable(UC, 0 <= p[i=1:Nbus] <= Pmax, Cont) # production
@variable(UC, 0<= d[i=1:Nbus] <= Dmax) # consumption
@variable(UC, z[i=1:Nbus], Bin) # generators' commitment (binary)
@variable(UC, -2*pi <= θ[i=1:Nbus] <= 2*pi) # phase angle

for i=1:Nbus
    @constraint(UC, p[i] + sum(NIM[i,l] * f[l] for l=1:Nbranches) == d[i]) # flow constraints
    @constraint(UC, p[i] >= z[i]*buses[i,:Pmin]) # min production
    @constraint(UC, p[i] <= z[i]*buses[i,:Pmax]) # max production

    @constraint(UC, d[i] <= buses[i, :Dmax])
end

for i=1:Nbranches
    @constraint(UC, f[i] == (1/branches[i,:x])*sum(-NIM[j,i]*θ[j] for j=1:Nbus)) # line flow expression, DC-approximation
end

@constraint(UC, θ[1] == 0) # slack node


@objective(UC, :Max, sum(buses[i,:b]*d[i] for i=1:Nbus) - sum(buses[j,:C]*p[j]+buses[j,:cSU]*z[j] for j=1:Nbus))

UC

Maximization problem with:
 * 16 linear constraints
 * 15 variables: 3 binary
Solver is CbcMathProg

### Model solution

In [7]:
status = solve(UC)
println("Objective value = ", getobjectivevalue(UC))

println("Actual demand = ", getvalue(d))
println("Supply = ", getvalue(p))
println("Commitment flag = ", getvalue(z))
println("theta = ", getvalue(θ))
println("flows = ", getvalue(f))

Objective value = 325.0
Actual demand = [0.0, 0.0, 7.0]
Supply = [2.0, 5.0, 0.0]
Commitment flag = [1.0, 1.0, 0.0]
theta = [0.0, 0.1, -0.3]
flows = [-1.0, 4.0, 3.0]
Presolve 1 (-15) rows, 3 (-12) columns and 3 (-30) elements
0  Obj -0 Dual inf 49.999999 (1)
1  Obj 325
Optimal - objective value 325
After Postsolve, objective 325, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 325 - 1 iterations time 0.002, Presolve 0.00
Cbc0045I Solution with objective value -325 saved


## An one-node test case
Porting the code from Python